---

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-10-18 04:50:14--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.10.78, 188.185.22.33, 188.185.33.206, ...
Connecting to zenodo.org (zenodo.org)|188.185.10.78|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2023-10-18 04:50:15--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

                     16%[==>                 ] 232.10M  23.9MB/s    eta 51s    ^C


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Datasets/SO_vectors_200.bin_download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
wv_embeddings.most_similar("dog")

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

Входит только cats (4 место), cat не входит

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [28]:
import numpy as np
import re
# you can use your tokenizer
# from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()
tokenizer.tokenize("how are you")

['how', 'are', 'you']

In [7]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question)
    question_embedding = np.zeros(dim)
    if not tokens:
      return question_embedding
    c = 0
    for word in tokens:
      if word in embeddings:
        c += 1
        question_embedding += embeddings[word]
    if c > 0:
      question_embedding /= c

    return question_embedding


In [ ]:
question_to_vec("i love chicken", wv_embeddings, tokenizer)

array([-0.61869673, -1.704759  , -0.45566978, -0.10751477,  0.38797171,
       -0.3701052 , -0.57228961, -2.11138284,  0.98081273,  0.45276632,
        0.76802061,  2.28003153, -0.18475629, -1.51086617,  0.34213456,
        0.92276126,  0.99592999,  0.46710166,  0.03954101,  2.21363276,
        1.50299436,  0.8681196 , -1.4939267 ,  0.27091449,  0.48541354,
       -0.89394495, -0.30649257, -0.59642637,  0.947148  , -1.22769569,
       -1.76258034, -1.02276438, -0.98898661, -1.90452236,  0.91741303,
       -0.0847283 ,  0.40733813,  0.29376012, -0.18833586,  0.71793076,
        1.15885988, -0.44339968, -1.07241043,  0.47554578,  0.01439353,
        0.70866705,  0.43209395,  0.34916985, -1.86546689,  0.52492758,
       -0.00938639, -0.26739222, -1.41111028,  0.38939266, -1.24047303,
        0.978043  ,  1.69989812, -0.92889777,  0.54173484, -1.67336869,
       -0.13001322, -1.55982453,  0.97182056,  0.15320546, -0.01001275,
       -0.10931379, -0.67115015,  0.555939  ,  0.39664747, -0.84

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
vec = question_to_vec("I love neural networks", wv_embeddings, tokenizer, wv_embeddings.vector_size)
round(vec[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?  
Ответ: 1, это максимально возможное из всех тк (1 или 0) - (1 или 0) = (-1 или 0 или 1). Например при значении rank_q = 46, N =1, ответ будет 1 - 0 = 1


<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)  
  
[K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}}$


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [6]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = sum(1 for rank in dup_ranks if rank <= k)
    return hits_value

In [8]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = sum([1 / np.log2(1 + rank) for rank in dup_ranks if rank <= k])
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [5]:
import pandas as pd
import numpy as np

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [9]:
!unzip /content/drive/MyDrive/Datasets/stackoverflow_similar_questions.zip

Archive:  /content/drive/MyDrive/Datasets/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [10]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split("\t"))
    return data

Нам понадобиться только файл validation.

In [11]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [13]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    vec = question_to_vec(question, embeddings, tokenizer)
    similarities = sorted([(i, cosine_similarity(vec.reshape(1, -1), question_to_vec(candidates[i], embeddings, tokenizer).reshape(1, -1))) for i in range(len(candidates))], key=lambda item: item[1], reverse = True)
    return [(s[0], candidates[s[0]]) for s in similarities]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(1, 'Getting all list items of an unordered list in PHP'), #скрыт
            (0, 'select2 not displaying search results'), #скрыт
            (2, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?  
Ответ 102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [14]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
ranks

[(284, 'Accessing libraries and functions between multiple source files'),
 (601,
  'How to use Aurelia third party plugin with without typescript definition file?'),
 (0, 'Include_in_parent using Nest elastic search library'),
 (292, 'plotGoogleMaps R package on CMS'),
 (624, 'Testing a Web application using AngularJS: MVC implementation'),
 (803, 'Local crash at /system/lib/libwebcore.so file'),
 (495, "How do I use Picasa's photo-editing features in my program?"),
 (630, 'VS 2010 setup shortcut file not shows Run as Administrator option'),
 (30, 'Webpack configuration for compiling module in node_modules'),
 (473, 'Python - what is the libs subfolder for?'),
 (211, 'Multi-module grails build'),
 (393,
  'Environment variable based runtime configuration with sbt native packager'),
 (152, 'How can I add opencv libraries to setup.py?'),
 (986, 'How to use shell32.dll from C++ console application'),
 (833, 'Importing an existing Eclipse android project into Android Studio'),
 (589,
  'T

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 285.000 | Hits@   1: 285.000
DCG@   5: 341.622 | Hits@   5: 393.000
DCG@  10: 359.693 | Hits@  10: 449.000
DCG@ 100: 405.681 | Hits@ 100: 679.000
DCG@ 500: 431.275 | Hits@ 500: 879.000
DCG@1000: 443.923 | Hits@1000: 1000.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [17]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [15]:
import re
from tqdm.notebook import tqdm

In [18]:
words = [re.sub(r'[^a-zA-Z0-9\s]', '', q).split() for d in train_data for q in d]

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, vector_size = 200,
                 min_count=5,
                 window=3).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 216.000 | Hits@   1: 216.000
DCG@   5: 277.113 | Hits@   5: 333.000
DCG@  10: 292.692 | Hits@  10: 382.000
DCG@ 100: 339.457 | Hits@ 100: 614.000
DCG@ 500: 372.600 | Hits@ 500: 874.000
DCG@1000: 385.935 | Hits@1000: 1000.000


In [ ]:
for j in range(1, 6, 2):
  embeddings_trained = Word2Vec(words, vector_size = 200,
                  min_count=5,
                  window=j).wv
  wv_ranking = []
  max_validation_examples = 1000
  for i, line in enumerate(tqdm(validation_data)):
      if i == max_validation_examples:
          break
      q, *ex = line
      ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
      wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  for k in tqdm([1, 5, 10, 100, 500, 1000]):
      print("Window = ", j)
      print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Window =  1
DCG@   1: 192.000 | Hits@   1: 192.000
Window =  1
DCG@   5: 243.607 | Hits@   5: 292.000
Window =  1
DCG@  10: 258.396 | Hits@  10: 338.000
Window =  1
DCG@ 100: 299.676 | Hits@ 100: 547.000
Window =  1
DCG@ 500: 335.033 | Hits@ 500: 828.000
Window =  1
DCG@1000: 353.141 | Hits@1000: 1000.000


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Window =  3
DCG@   1: 209.000 | Hits@   1: 209.000
Window =  3
DCG@   5: 270.910 | Hits@   5: 323.000
Window =  3
DCG@  10: 289.941 | Hits@  10: 382.000
Window =  3
DCG@ 100: 337.013 | Hits@ 100: 616.000
Window =  3
DCG@ 500: 369.734 | Hits@ 500: 873.000
Window =  3
DCG@1000: 383.183 | Hits@1000: 1000.000


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Window =  5
DCG@   1: 232.000 | Hits@   1: 232.000
Window =  5
DCG@   5: 292.467 | Hits@   5: 347.000
Window =  5
DCG@  10: 311.213 | Hits@  10: 405.000
Window =  5
DCG@ 100: 361.900 | Hits@ 100: 654.000
Window =  5
DCG@ 500: 392.129 | Hits@ 500: 890.000
Window =  5
DCG@1000: 403.796 | Hits@1000: 1000.000


Попробуем другие токенизаторы

In [ ]:
import nltk
from nltk.tokenize import WordPunctTokenizer
nltk.download('punkt')
nltkTokenizer = WordPunctTokenizer()
words_nltk = [nltkTokenizer.tokenize(re.sub(r'[^a-zA-Z0-9\s]', '', q)) for d in train_data for q in d]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
embeddings_trained_nltk = Word2Vec(words_nltk, vector_size = 200,
                  min_count=5,
                window=5).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained_nltk, nltkTokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("NLTK tokenizer")
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

NLTK tokenizer
DCG@   1: 228.000 | Hits@   1: 228.000
NLTK tokenizer
DCG@   5: 289.205 | Hits@   5: 345.000
NLTK tokenizer
DCG@  10: 309.625 | Hits@  10: 408.000
NLTK tokenizer
DCG@ 100: 359.062 | Hits@ 100: 652.000
NLTK tokenizer
DCG@ 500: 389.394 | Hits@ 500: 888.000
NLTK tokenizer
DCG@1000: 401.278 | Hits@1000: 1000.000


Результат стал немного хуже, то есть в данном случае токенизация по пробелу работает лучше токенизатора из библиотеки nltk

In [20]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [26]:
words_lemmatized =[[lemmatizer.lemmatize(w) for w in sent] for sent in words]

In [27]:
from gensim.models import Word2Vec
embeddings_trained_lemma = Word2Vec(words_lemmatized, vector_size = 200,
                  min_count=5,
                window=5).wv

In [29]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained_lemma, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 207.000 | Hits@   1: 207.000
DCG@   5: 268.530 | Hits@   5: 325.000
DCG@  10: 287.202 | Hits@  10: 383.000
DCG@ 100: 337.673 | Hits@ 100: 633.000
DCG@ 500: 368.581 | Hits@ 500: 876.000
DCG@1000: 381.716 | Hits@1000: 1000.000


Лемматизация ухудшила метрики при низких К и улучшила при высоких К.

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Выводы:


# В данной задаче токенизация по пробелу работает лучше токенизатора из библиотеки nltk (WordPunctTokenizer)

# На этих данных лемматизация ухудшает метрики при низких К и улучшает при высоких К. То есть можно сказать что она усредняет общий результат

# Эмбеддинги взятые со Stack Overflow лучше работают в данном кейсе. Значит что текста на которых они были обучены больше подходжят под наши данные

# Плохое качество результатов может быть по разным причинам, например обучающие и валидационные текста разные и из-за этого эмбединги строятся неподходящие.